In [1]:
!pip install -q transformers accelerate bitsandbytes huggingface_hub rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.5 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/chaineypung/Pytorch-Swin-Unet-V2.git
!mv Pytorch-Swin-Unet-V2/network .
!rm -rf Pytorch-Swin-Unet-V2

Cloning into 'Pytorch-Swin-Unet-V2'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 39 (delta 15), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 117.61 KiB | 1.59 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [3]:
import os
import uuid
import argparse
from datetime import datetime

import numpy as np
import torch
import torch.nn.functional as F
import rasterio
from skimage.measure import label, regionprops
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline, BitsAndBytesConfig
)

In [ ]:
# Initializing model for inference
from network.swin_unet_v2 import SwinTransformerSys

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
REFLECT_SCALE = 1e4
IMG_SIZE = (224, 224)
TARGET_SHAPE = (351, 303)
PIXEL_AREA_KM2 = (30 * 30) / 1e6

# Initialize vision model
swin_unet_model = SwinTransformerSys(
    img_size=IMG_SIZE[0], patch_size=4, in_chans=23, num_classes=23,
    embed_dim=96, depths=[2,2,2,2], depths_decoder=[1,2,2,2],
    num_heads=[3,6,12,24], window_size=7, mlp_ratio=4.0,
    drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.1,
    norm_layer=torch.nn.LayerNorm, final_upsample="expand_first"
).to(DEVICE)
swin_unet_model.load_state_dict(torch.load("/content/best_swin_v2.pth", map_location=DEVICE))
swin_unet_model.eval()

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:23
---final upsample expand_first---


SwinTransformerSys(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(23, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=96, input_resolution=(56, 56), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=96, input_resolution=(56, 56), num_heads=3, window_size=7, shift_size=0, mlp_ratio=4.0
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            dim=96, window_size=(7, 7), num_heads=3
            (cpb): Mlp_Relu(
              (fc1): Linear(in_features=2, out_features=256, bias=True)
              (fc2): Linear(in_features=256, out_features=3, bias=True)
              (act): ReLU()
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_

In [5]:
# Function to preprocess the input
def preprocess_tiff(input_tif: str, output_npy: str):
    with rasterio.open(input_tif) as src:
        if src.count != 23:
            raise ValueError(f"Expected 23 bands, got {src.count}")
        img = src.read()

    nan_ratio = np.isnan(img).sum() / img.size
    if nan_ratio > 0.5:
        raise ValueError(f"Too many NaNs: {nan_ratio:.1%}")

    img = np.nan_to_num(img, nan=0.0)
    img = np.clip(img, a_min=0.0, a_max=None)

    if np.all(img == 0):
        raise ValueError("Image is entirely zero")

    b, h, w = img.shape
    th, tw = TARGET_SHAPE
    top = max((h - th)//2, 0)
    left = max((w - tw)//2, 0)
    img2 = img[:, top:top+th, left:left+tw]

    pad_h = max(th - img2.shape[1], 0)
    pad_w = max(tw - img2.shape[2], 0)
    pt, pb = pad_h//2, pad_h - pad_h//2
    pl, pr = pad_w//2, pad_w - pad_w//2
    aligned = np.pad(
        img2,
        ((0,0), (pt,pb), (pl,pr)),
        mode='constant', constant_values=0
    )

    np.save(output_npy, aligned)

In [ ]:
def infer_and_metrics_npy(npy_path: str):
    arr = np.load(npy_path)
    # Day t true mask and area
    mask_t = (arr[0] / REFLECT_SCALE > 0).astype(np.uint8)
    n_t = mask_t.sum()
    area_t = n_t * PIXEL_AREA_KM2

    # Model inference
    refl = arr
    H, W = refl.shape[1:]
    x0 = torch.from_numpy(refl / REFLECT_SCALE).unsqueeze(0).float().to(DEVICE)
    x_small = F.interpolate(x0, size=IMG_SIZE, mode="bilinear", align_corners=False)
    with torch.no_grad():
        out_small = swin_unet_model(x_small)
        out_up = F.interpolate(out_small, size=(H, W), mode="bilinear", align_corners=False)[0]

    # Day t+1 predicted mask amd area
    fire_prob = torch.sigmoid(out_up[0]).cpu().numpy()
    mask_pred = (fire_prob > 0.5).astype(np.uint8)
    n_pred = mask_pred.sum()
    area_pred = n_pred * PIXEL_AREA_KM2

    # Increase in fire
    delta_n = n_pred - n_t
    delta_area = area_pred - area_t

    # Hotspots
    idxs = np.argpartition(-fire_prob.flatten(), 5)[:5]
    ys, xs = np.unravel_index(idxs, fire_prob.shape)
    hotspots = [{"x":int(x),"y":int(y),"p":float(fire_prob[y,x])} for x,y in zip(xs,ys)]

    # False-color and corridor
    rgb0 = np.clip(arr[[3,2,1]].transpose(1,2,0)/REFLECT_SCALE,0,1)
    pred_ref = out_up.cpu().numpy() * REFLECT_SCALE
    rgb_pred = np.clip(pred_ref[[3,2,1]].transpose(1,2,0)/REFLECT_SCALE,0,1)
    diff = rgb0.mean(2) - rgb_pred.mean(2)
    th = diff.mean() + diff.std()
    corridor = diff > th

    # Fire direction
    ys_t, xs_t = np.nonzero(mask_t)
    ys_c, xs_c = np.nonzero(corridor)
    if len(xs_t) and len(xs_c):
        c0 = np.array([xs_t.mean(), ys_t.mean()])
        c1 = np.array([xs_c.mean(), ys_c.mean()])
        dx, dy = c1 - c0
        direction = ("S" if dy > 0 else "N") + ("E" if dx > 0 else "W")
    else:
        direction = None

    return {
        "n_t": n_t, "area_t": area_t,
        "n_pred": n_pred, "area_pred": area_pred,
        "delta_n": delta_n, "delta_area": delta_area,
        "hotspots": hotspots, "direction": direction
    }

In [ ]:
# Configuration for LLM
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2"
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Tokenizer and model for text generation
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_auth_token=token)
quant_cfg = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)
llm_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quant_cfg,
    device_map="auto",
    torch_dtype="auto",
    use_auth_token=token
)
pipeline = TextGenerationPipeline(model=llm_model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# Prompt for LLM model
def build_prompt_fixed(m: dict, pct: float, speed: float, rpt_date: str) -> str:
    rows = "\n".join(
        f"| {i+1} | {h['x']} | {h['y']} | {h['p']:.3f} |"
        for i, h in enumerate(m["top_5_hotspots"]))
    return f"""
You are an expert wildfire incident analyst generating an operational briefing for the fire department.
**Report Date:** {rpt_date}

**Do not repeat the prompt. Only output the report in Markdown.**

---

## 0. Report Date
- **{rpt_date}**

## 1. Executive Summary
- Yesterday’s fire burned **{m['day_t_true_fire_area']}**, and it's expected to grow to **{m['day_t1_predicted_fire_area']}**.
- The area increased by **{m['increase_in_fire_area']}**, a **{pct:.2f}%** rise in 24 h.
- The average spread rate is **{speed:.2f} km²/hr**.
- The main direction of spread is **{m['direction_of_spread']}**.

---

## 2. Key Metrics
| Metric                   | Value                         |
|--------------------------|-------------------------------|
| Yesterday’s burn area    | {m['day_t_true_fire_area']}   |
| Tomorrow’s predicted     | {m['day_t1_predicted_fire_area']} |
| Increase in fire area    | {m['increase_in_fire_area']}  |
| Percent increase         | {pct:.2f}%                    |
| Avg. spread speed        | {speed:.2f} km²/hr            |
| Spread direction         | {m['direction_of_spread']}    |

---

## 3. Top-5 Hotspots
| # | X   | Y   | Confidence |
|:-:|:---:|:---:|:-----------:|
{rows}

---

## 4. Actionable Recommendations
- Establish defensive lines along the **{m['direction_of_spread']}** perimeter to slow further spread.
- Evacuate at-risk zones immediately {m['direction_of_spread'].lower()} of the fire front.
- Pre-position medical teams at staging area α (closest to predicted growth corridor).
- Deploy additional aerial water drops focused on high-confidence hotspots.

---

*End of report.*
""".strip()

In [10]:
sample_input = "/content/2018-01-01.tif"

uid = uuid.uuid4().hex
npy_path = f"{uid}.npy"

preprocess_tiff(sample_input, npy_path)
res = infer_and_metrics_npy(npy_path)

api_metrics = {
    "day_t_true_fire_area": f"{res['area_t']:.2f} km²",
    "day_t1_predicted_fire_area": f"{res['area_pred']:.2f} km²",
    "increase_in_fire_area": f"{res['delta_area']:.2f} km²",
    "top_5_hotspots": res["hotspots"],
    "direction_of_spread": res["direction"]
    }

true_area = float(api_metrics["day_t_true_fire_area"].split()[0])
delta = float(api_metrics["increase_in_fire_area"].split()[0])
percent_increase = (delta / true_area) * 100
avg_speed = delta / 24

report_date = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")
prompt = build_prompt_fixed(api_metrics, percent_increase, avg_speed, report_date)
print("\n---PROMPT---\n", prompt, "\n---END PROMPT---\n")

output = pipeline(
    prompt,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

print("---GENERATED REPORT---\n")
print(output[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



---PROMPT---
 You are an expert wildfire incident analyst generating an operational briefing for the fire department.
**Report Date:** 2025-05-01 17:22 UTC

**Do not repeat the prompt. Only output the report in Markdown.**

---

## 0. Report Date
- **2025-05-01 17:22 UTC**

## 1. Executive Summary
- Yesterday’s fire burned **67.58 km²**, and it's expected to grow to **72.99 km²**.
- The area increased by **5.41 km²**, a **8.01%** rise in 24 h.
- The average spread rate is **0.23 km²/hr**.
- The main direction of spread is **SE**.

---

## 2. Key Metrics
| Metric                   | Value                         |
|--------------------------|-------------------------------|
| Yesterday’s burn area    | 67.58 km²   |
| Tomorrow’s predicted     | 72.99 km² |
| Increase in fire area    | 5.41 km²  |
| Percent increase         | 8.01%                    |
| Avg. spread speed        | 0.23 km²/hr            |
| Spread direction         | SE    |

---

## 3. Top-5 Hotspots
| # | X   | Y   | 

/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


---GENERATED REPORT---

You are an expert wildfire incident analyst generating an operational briefing for the fire department.
**Report Date:** 2025-05-01 17:22 UTC

**Do not repeat the prompt. Only output the report in Markdown.**

---

## 0. Report Date
- **2025-05-01 17:22 UTC**

## 1. Executive Summary
- Yesterday’s fire burned **67.58 km²**, and it's expected to grow to **72.99 km²**.
- The area increased by **5.41 km²**, a **8.01%** rise in 24 h.
- The average spread rate is **0.23 km²/hr**.
- The main direction of spread is **SE**.

---

## 2. Key Metrics
| Metric                   | Value                         |
|--------------------------|-------------------------------|
| Yesterday’s burn area    | 67.58 km²   |
| Tomorrow’s predicted     | 72.99 km² |
| Increase in fire area    | 5.41 km²  |
| Percent increase         | 8.01%                    |
| Avg. spread speed        | 0.23 km²/hr            |
| Spread direction         | SE    |

---

## 3. Top-5 Hotspots
| # | X  

### Reference

- https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2
- https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.TextGenerationPipeline
- https://github.com/chaineypung/Pytorch-Swin-Unet-V2
- https://huggingface.co/docs/transformers/en/quantization/bitsandbytes
- https://medium.com/@rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996